In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import cv2
import glob
import os
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
#from keras.initializations import norRemal, identity
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.optimizers import RMSprop, Adadelta
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Dense, Activation, TimeDistributedDense, Dropout, Reshape, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.models import model_from_json

Using TensorFlow backend.


In [195]:
def load_csv(n_rows=None):
    file = pd.read_csv("data/capture_inferno_validation.csv", nrows=n_rows)
    file.columns = ['index', 'steer', 'gear', 'accel', 'brake', 'clutch', 'speed']
    file = file[['index', 'speed']]
    file['index'] = file['index'].apply(lambda x: str(x) + ".png")
    return(file)

In [196]:
def get_im_cv(path, img_rows, img_cols, color_type=1):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    return resized

In [197]:
def load_images(raw_images, dataset_csv):
    images = []
    speeds = []
    for raw_image in raw_images:
        img = get_im_cv(raw_image, 66, 100, 3)
        raw_image_name = raw_image.split('/')[-1]
        speed = dataset_csv['speed'][dataset_csv['index'] == raw_image_name]
        images.append(img)
        speeds.append(float(speed.values))
        
    return(np.array(images), np.array(speeds))
        

In [198]:
def create_model(maxToAdd,size):
    model = Sequential()
    model.add(TimeDistributed(Convolution2D(24, 3, 3, border_mode='valid'), input_shape=(maxToAdd,size,size, 1)))
    model.add(Activation('relu'))
    
    model.add(TimeDistributed(Convolution2D(36, 3, 3, border_mode='valid'), input_shape=(maxToAdd,size,size, 1)))
    model.add(Activation('relu'))
    
    model.add(TimeDistributed(Convolution2D(64, 3, 3, border_mode='valid'), input_shape=(maxToAdd,size,size, 1)))
    model.add(Activation('relu'))
    
    model.add(TimeDistributed(Convolution2D(64, 3, 3, border_mode='valid'), input_shape=(maxToAdd,size,size, 1)))
    model.add(Activation('relu'))
    
    
#     model.add(TimeDistributed(Convolution2D(16, 2, 2, border_mode='valid')))
#     #model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),border_mode='valid')))
#     #model.add(Activation('relu'))
#     #model.add(TimeDistributed(Convolution2D(8, 3, 3, border_mode='valid')))
#     model.add(Activation('relu'))
    #model.add(Reshape((maxToAdd,np.prod(model.output_shape[-3:])))) #this line updated to work with keras 1.0.2
    model.add(TimeDistributed(Flatten()))
    model.add(Activation('relu'))
    model.add(GRU(output_dim=500,return_sequences=True))
    model.add(GRU(output_dim=100,return_sequences=True))
    model.add(GRU(output_dim=50,return_sequences=False))
    model.add(Dropout(.2))
    model.add(Dense(1))

    rmsprop = RMSprop()
    model.compile(loss='mean_squared_error', optimizer=rmsprop)
    return(model)

In [199]:
raw_images = glob.glob("data/images_validation/*.png")

In [200]:
csv_file = load_csv()

In [201]:
images, speeds = load_images(raw_images, csv_file)
images = images[:, :, :, :1]
images = images.astype('float32')
images /= 255

In [185]:
# plt.imshow(images[0])
# plt.imshow(images[0].reshape((100, 100)))

In [202]:
batch_size      = 32
nb_epochs       = 20
examplesPer     = 2000
maxToAdd        = 20
hidden_units    = 200
rows            = 66
cols            = 100
X_train       = []
y_train       = []
X_test        = []
y_test        = []    

numToAdd = maxToAdd
for i in range(0,examplesPer):

    X_train.append(images[i:i+numToAdd])
    spd = speeds[i+numToAdd-1]

    y_train.append(spd)
X_train = np.array(X_train)
y_train = np.array(y_train)


In [ ]:
filepath="weights/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='min')
logger = CSVLogger("overview.csv", separator=',', append=True)
callbacks_list = [checkpoint, logger]

In [189]:
model = create_model(maxToAdd, size)

In [190]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=100, nb_epoch= 100, callbacks=callbacks_list)

Epoch 1/10
1216/2000 [=================>............] - ETA: 143s - loss: 2451.4026

KeyboardInterrupt: 